## Checking IP address

In [ ]:
!curl ipecho.net/plain

35.188.238.117

In [ ]:
# code adopted from https://andrew-muller.medium.com/scraping-steam-user-reviews-9a43f9e38c92

# importing the packages. 
from google.colab import drive
from tqdm import tqdm
drive.mount("/content/drive")
%cd '/content/drive/My Drive/Dissertation'
import requests
from bs4 import BeautifulSoup

# this collects the app_ids for all single-player FPS games 
def get_n_appids(tag_no, filter_by='topsellers',n=1000): # according to steam there are about ~600 FPS games so we just set n=1000 here 
    appids = []
    url = f'https://store.steampowered.com/search/?category1=998&tags={tag_no}&filter={filter_by}&page='
    page = 0
    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            appids.append(row['data-ds-appid'])
            #print(len(appids))

    return appids[:n]

single_FPS_appids = get_n_appids(tag_no="1663%2C4182") # "1663%2C4182" is the singleplayer-FPS tag


# This get's us 1 review
def get_one_review(appid, params={'json':1}):
        url = 'https://store.steampowered.com/appreviews/'
        response = requests.get(url=url+appid, params=params, headers={'User-Agent': 'Mozilla/5.0'})
        return response.json()

# get all reviews from 1 specific game 
def get_all_reviews(appid):
    review_count=int(((get_one_review(str(appid)))["query_summary"])["total_reviews"])
    reviews = []
    cursor = '*'
    params = {
            'json' : 1,
            'filter' : 'all',
            'language' : 'english',
            'day_range' : 9223372036854775807,
            'review_type' : 'all',
            'purchase_type' : 'all'
            }
    while review_count > 0:
        params['cursor'] = cursor.encode()
        params['num_per_page'] = min(100, review_count)
        review_count -= 100
        response = get_one_review(appid, params)
        cursor = response['cursor']
        reviews += response['reviews']
        if len(response['reviews']) < 100: 
          break
    return (reviews)
# this collects all the reviews across a list of game ids. 
def collect_reviews(FPS_appids):
  review_list=[]
  for i in tqdm(FPS_appids):
    appid=str(i)
    reviews=get_all_reviews(appid)
    review_list.append(reviews)
  return(review_list)

len(single_FPS_appids)

Mounted at /content/drive
/content/drive/My Drive/Dissertation


673

In [ ]:
# saving as a pickle file
# saving games_list
import pickle
with open("games_list_2","wb") as fp:
  pickle.dump(single_FPS_appids, fp)

In [ ]:
# collating and saving game reviews
single_FPS_reviews=collect_reviews(single_FPS_appids)
import pickle

with open("singleplayer_FPS_reviews_list_2", "wb") as fp:
  pickle.dump(single_FPS_reviews, fp)

100%|██████████| 673/673 [2:20:09<00:00, 12.50s/it]
